## Imports

In [ ]:
import numpy as np
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar100

## Preprocessing



### Load dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 3s 0us/step


### Data normalization

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


## Learning

### Building Resnet



In [ ]:
def identity_block(X, filters):
    
  # Retrieve Filters
  F1, F2, F3 = filters

  # Save the input value. You'll need this later to add back to the main path. 
  X_shortcut = X

  # First component of main path
  X = tf.keras.layers.Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
  X = tf.keras.layers.BatchNormalization()(X)
  X = tf.keras.layers.Activation('relu')(X)

  # Second component of main path
  X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(3,3), strides=(1,1), padding='same')(X)
  X = tf.keras.layers.BatchNormalization()(X)
  X = tf.keras.layers.Activation('relu')(X)

  # Third component of main path
  X = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
  X = tf.keras.layers.BatchNormalization()(X)

  # Final step: Add shortcut value to main path, and pass it through a RELU activation
  X = tf.keras.layers.Add()([X,X_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X

In [ ]:
def convolutional_block(X, filters, s):
  # Retrieve Filters
  F1, F2, F3 = filters
    
  # Save the input value. You'll need this later to add back to the main path. 
  X_shortcut = X

  ##### MAIN PATH #####
  # First component of main path 
  X = tf.keras.layers.Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding='valid')(X)
  X = tf.keras.layers.BatchNormalization()(X)
  X = tf.keras.layers.Activation('relu')(X)

  # Second component of main path
  X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(3,3), strides=(1,1), padding='same')(X)
  X = tf.keras.layers.BatchNormalization()(X)
  X = tf.keras.layers.Activation('relu')(X)

  # Third component of main path
  X = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
  X = tf.keras.layers.BatchNormalization()(X)

  ##### SHORTCUT PATH ####
  X_shortcut = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid')(X_shortcut)
  X_shortcut = tf.keras.layers.BatchNormalization()(X_shortcut)

  # Final step: Add shortcut value to main path, and pass it through a RELU activation
  X = tf.keras.layers.Add()([X,X_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X

In [ ]:
X_input = tf.keras.layers.Input((32,32,3))
X = tf.keras.layers.experimental.preprocessing.Resizing(224,224)(X_input)

# Stage 1
X = tf.keras.layers.ZeroPadding2D((3,3))(X)
X = tf.keras.layers.Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='valid')(X)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.ZeroPadding2D((1,1))(X)
X = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2))(X)

# Stage 2
X = convolutional_block(X, [64,64,256], 1)
X = identity_block(X, [64,64,256])
X = identity_block(X, [64,64,256])

# Stage 3
X = convolutional_block(X, [128,128,512], 2)
X = identity_block(X, [128,128,512])
X = identity_block(X, [128,128,512])
X = identity_block(X, [128,128,512])

# Stage 4 
X = convolutional_block(X, [256,256,1024], 2)
X = identity_block(X, [256,256,1024])
X = identity_block(X, [256,256,1024])
X = identity_block(X, [256,256,1024])
X = identity_block(X, [256,256,1024])
X = identity_block(X, [256,256,1024])

# Stage 5 
X = convolutional_block(X, [512,512,2048], 2)
X = identity_block(X, [512,512,2048])
X = identity_block(X, [512,512,2048])

# AVGPOOL
X = tf.keras.layers.GlobalAveragePooling2D()(X)

# output layer
X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.Dense(100, activation='softmax')(X)

# Create model
model = tf.keras.models.Model(inputs=X_input, outputs=X, name='ResNet50')

opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
resizing_1 (Resizing)           (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           resizing_1[0][0]                 
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 112, 112, 64) 9472        zero_padding2d_2[0][0]           
___________________________________________________________________________________________

### Train

In [ ]:
history = model.fit(X_train, y_train, batch_size=100, validation_split=0.1, epochs=5)

Epoch 1/5


KeyboardInterrupt: ignored

### Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(X_train, y_train, batch_size=100)
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=100)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()